In [1]:
import sys
import os
# initialize to make sure project root is in the path
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from adviser.smart_traveler_info_retriver import (
    construct_chain_for_getting_advice_url,
    construct_chain_for_advice_doc_from_url,
    advice_whether_safe_to_travel,
    get_tool_calling_model,
    )
from adviser.advise_model import create_prompt_for_travel_advice_response
    

In [3]:
query = 'I would like to travel to Papua. Is it safe??'

In [4]:
chat_model = ChatOpenAI(temperature=0, model="gpt-4o-mini-2024-07-18")

In [5]:
chain_tool = get_tool_calling_model(chat_model=chat_model, calling_tools=[advice_whether_safe_to_travel])

In [6]:
chain_url = construct_chain_for_getting_advice_url(chat_model=chat_model, 
                                               calling_tool=advice_whether_safe_to_travel,)

In [7]:
chain_url.invoke({'query': query})

'https://www.smartraveller.gov.au/destinations/asia/indonesia'

In [14]:
chain =  (RunnableParallel({"doc": chain_url | construct_chain_for_advice_doc_from_url(),
                               "query": RunnablePassthrough()})
                              | create_prompt_for_travel_advice_response
                              | chat_model
                              | StrOutputParser())

In [22]:
query = 'I would like to travel to guangzhou. Is it safe??'
print(chain.invoke({'query': query}))

Travel Safety Level: 
    "Exercise a high degree of caution" in China overall.
Reasons:
    - Authorities have detained foreigners on the grounds of 'endangering national security'.
    - Australians may be at risk of arbitrary detention or harsh enforcement of local laws, including broadly defined National Security Laws.
    - The security situation in regions like Tibet and Xinjiang necessitates careful planning and precautions. 

Since no specific advice level for Guangzhou is provided, the overall advice for China applies.


In [18]:
query = 'I would like to travel to papua. Is it safe??'
print(chain.invoke({'query': query}))

Travel Safety Level: 
"Exercise a high degree of caution" in Indonesia overall.
"Reconsider Your Need to Travel" in Papua.

Reasons:
- Ongoing security risks in Indonesia necessitate careful planning and precautions.
- There is a risk of serious security incidents or demonstrations that may turn violent in Papua.
